In [ ]:
!git clone https://github.com/rafaseto/PIBIC-AnaliseDados.git

In [ ]:
%cd PIBIC-AnaliseDados

In [ ]:
import pip

In [ ]:
!pip install dlisio

## Importa funções para o pré-processamento

In [ ]:
from aux_datapreprocess import *

In [1]:
from dlisio import dlis
import pandas as pd

In [2]:
# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carrega os dados

In [6]:
dli_dict = {}        # Conterá os arquivos lógicos
filenames = []  # Conterá os caminhos dos arquivos dlis
nomes_anp = []      # Conterá os nomes ANP dos poços
charId_inicio, charId_fim = 7, 10   # Índices de início e fim da substring identificadora do poço

# Utilizando a função 'glob' do módulo 'glob' para procurar todos os arquivos com extensão DLIS em 'Data'
for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    
    # Carregando os arquivos dlis com 'load' e armazenando-os em 'leitura'
    # 'tail' recebe valores restantes, caso a função retorne mais de uma peça de informação
    leitura, *tail = dlis.load(f'{file}')
    
    nome = leitura.origins[0].well_name

    nomes_anp.append(nome)    # armazenando o nome do poço
    
    filenames.append(file)      # armazenando o caminho dos arquivos dlis
    
    nome_abreviado = nome[charId_inicio : charId_fim]   # identificador do poço, e.g. '900'
    
    # Armazenando o arquivo lógico como valor em um dicionário onde a chave é o identificador do poço
    dli_dict[nome_abreviado] = leitura      

In [7]:
dli_dict['900'].describe()

------------
Logical File
------------
Description : LogicalFile(GEOLOAD.1)
Frames      : 1
Channels    : 21

Known objects
--
FRAME       : 1
TOOL        : 1
PARAMETER   : 167
CHANNEL     : 21
NO-FORMAT   : 1
FILE-HEADER : 1
ORIGIN      : 1
COMMENT     : 2

Unknown objects
--
280-FRAMESTEP-INFO : 1


## Cria dataframes para os poços

In [8]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Curvas de perfis que não serão utilizadas
curvas_nao_utilizadas = ['WF11', 'AHV', 'AHVT', 'BHV', 'BHVT', 'CS', 'FCPS', 'HDCN', 'HMCN', 'ITT', 'ITTT', 'LSPD', 'MMK', 'NCPS', 'SP', 'TENS']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():
    try:
        # Armazenando as curvas que serão utilizadas em uma lista
        curvas_utilizadas = [
            channel.name                                    # Os elementos da lista serão os nomes das curvas
            for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
            if channel.name not in curvas_nao_utilizadas    # As curvas que não utilizaremos não serão armazenadas na lista
        ]


        curvas = poco.frames[0].curves()

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df_dict[chave] = pd.DataFrame(curvas[curvas_utilizadas])
    except:
        pass

In [9]:
# Transformados os valores -999.25 em nulos
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Remoção de Colunas

In [ ]:
colunas_para_remover = ['CALI', 'RHOB', 'DRHO', 'PE', 'HDRS', 'HMRS', 'DTC']
dlis_df_dict['900'] = remove_colunas(dlis_df_dict['900'], colunas_para_remover)

In [ ]:
dlis_df_dict['900']

## Renomeação de Colunas

In [11]:
dlis_df_dict['900'] = renomeiaColuna(dlis_df_dict['900'], "GR", "gamma-ray")

In [12]:
dlis_df_dict['900']

,TDEP,gamma-ray,BS,NPHI,CALI,RHOB,DRHO,PE,HDRS,HMRS,DTC
0,374.899994,None,8.5,None,None,None,None,None,None,None,None
1,374.799994,None,8.5,None,None,None,None,None,None,None,None
2,374.699994,None,8.5,None,None,None,None,None,None,None,None
3,374.599994,None,8.5,None,None,None,None,None,None,None,None
4,374.499994,None,8.5,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
3595,15.399989,55.795601,8.5,None,None,None,None,None,None,None,None
3596,15.299989,52.5182,8.5,None,None,None,None,None,None,None,None
3597,15.199989,48.504501,8.5,None,None,None,None,None,None,None,None
3598,15.099989,45.021198,8.5,None,None,None,None,None,None,None,None


## Transformação NPHI

In [ ]:
dlis_df_dict['897']

In [ ]:
nphi_transform(dlis_df_dict, ['900', '897'])

In [ ]:
dlis_df_dict['897']

## Remoção de Pontos com Falha

calcular limites para DCALI (2 polegadas, se for maior descartar), DRHO (-0.15, +0.15) --> nesse intervalo, fora descarta

In [ ]:
dlis_df_dict['897']

In [ ]:
pontos_com_falha = {
    '900': [250, 360],
    '889': [75, 210],
    '912': [60, 260],
    '897': [90, 221],
    '910': [64, 215],
    '907': [90, 320]
}

In [ ]:
#dlis_df_dict['1847'].drop(dlis_df_dict['1847'][dlis_df_dict['1847']['TDEP'] > 300].index)

In [ ]:
# Chamando a função
remove_pontos_com_falha(dlis_df_dict, pontos_com_falha)

## Adiciona coluna DCALI

In [13]:
add_DCALI(dlis_df_dict)

In [14]:
limita_curva(dlis_df_dict, "DCALI", -2, 2)

## Remove valores DRHO indesejados

In [15]:
limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)

In [ ]:
pd.set_option('display.max_rows', None)
dlis_df_dict['897']